# Creating Knowledge graphs from Pdf files

In [2]:
from langchain.graphs import Neo4jGraph

url = ""
username =""
password = ""
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [15]:
# Add OpenAI key
os.environ['OPENAI_API_KEY']=''

In [4]:
from langchain_community.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

# class Source(BaseSource):
#     properties: Optional[List[Property]] = Field(
#         None, description="List of sources"
#     )        
        
class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

In [5]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

# Using GPT-3.5-turbo-16k model to create the knowledge graph

In [16]:
import os
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# os.environ["OPENAI_API_KEY"] = "sk-"
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
        [(
          "system",
          f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
          """),
            ("human", "Use the given format to extract information from the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

In [45]:
def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.invoke(document.page_content)['function']
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    # Store information into a graph
    print(graph_document)
    graph.add_graph_documents([graph_document])
    return graph_document

In [77]:

from langchain.text_splitter import TokenTextSplitter

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/December_2023_Bolt.pdf")

pages = loader.load_and_split()

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

# Only take the first 4 pages of the document
documents = text_splitter.split_documents(pages[:4])

In [78]:
documents

[Document(page_content='oliv ebo a r d.in uick revisionCLICK HERE TO PREPARE FOR \nIBPS, SSC, SBI, RAILWAYS & RBI EXAMS IN ONE PLACE\nBolt is a series of GK Summary ebooks by  Oliveboard  for q\noliveboard\nNow with\nIn-depth\nCoverageDecember 2023\nMonthly Current Affairs', metadata={'source': 'C:/Users/rakshita_arora/Downloads/December_2023_Bolt.pdf', 'page': 0}),
 Document(page_content='www.oliveboard.in  \nwww.oliveboard.in  \nTable of Contents  \nBanking & Finance  ................................ ................................ ................................ ..................  3 \nEconomy  ................................ ................................ ................................ ...............................  15 \nNational  ................................ ................................ ................................ ................................  24 \nInternational  ................................ ................................ ...........................

In [79]:
from tqdm import tqdm

for i, d in tqdm(enumerate(documents), total=len(documents)):
    graph_document=extract_and_store_graph(d)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

nodes=[Node(id='Oliveboard', type='Organization', properties={'name': 'Oliveboard'})] relationships=[] source=Document(page_content='oliv ebo a r d.in uick revisionCLICK HERE TO PREPARE FOR \nIBPS, SSC, SBI, RAILWAYS & RBI EXAMS IN ONE PLACE\nBolt is a series of GK Summary ebooks by  Oliveboard  for q\noliveboard\nNow with\nIn-depth\nCoverageDecember 2023\nMonthly Current Affairs', metadata={'source': 'C:/Users/rakshita_arora/Downloads/December_2023_Bolt.pdf', 'page': 0})


 25%|█████████████████████                                                               | 1/4 [00:01<00:04,  1.65s/it]

nodes=[Node(id='Www.Oliveboard.In', type='Website', properties={'name': 'Www.Oliveboard.In'}), Node(id='Banking & Finance', type='Category', properties={'name': 'Banking & Finance'}), Node(id='Economy', type='Category', properties={'name': 'Economy'}), Node(id='National', type='Category', properties={'name': 'National'}), Node(id='International', type='Category', properties={'name': 'International'}), Node(id='Sports', type='Category', properties={'name': 'Sports'}), Node(id='Agreements & Mous', type='Category', properties={'name': 'Agreements & Mous'}), Node(id='Schemes', type='Category', properties={'name': 'Schemes'}), Node(id='Appointments & Resignations', type='Category', properties={'name': 'Appointments & Resignations'}), Node(id='Awards', type='Category', properties={'name': 'Awards'}), Node(id='Ranks & Reports', type='Category', properties={'name': 'Ranks & Reports'}), Node(id='Important Days', type='Category', properties={'name': 'Important Days'}), Node(id='Summit & Conferen

 50%|██████████████████████████████████████████                                          | 2/4 [00:14<00:16,  8.02s/it]

nodes=[Node(id='Www.Oliveboard.In', type='Website', properties={'name': 'Www.Oliveboard.In'}), Node(id='Bank Of America', type='Bank', properties={'ceo': 'Brian Thomas Moynihan', 'name': 'Bank Of America'}), Node(id='Hdfc Bank', type='Bank', properties={'name': 'Hdfc Bank'}), Node(id='Rbi', type='Organization', properties={'name': 'Rbi'}), Node(id='Liberalised Remittance Scheme', type='Scheme', properties={'year': '1999', 'name': 'Liberalised Remittance Scheme'}), Node(id='Sbi Mutual Fund', type='Mutual fund', properties={'name': 'Sbi Mutual Fund'}), Node(id='Karur Vysya Bank', type='Bank', properties={'name': 'Karur Vysya Bank'})] relationships=[Relationship(source=Node(id='Rbi', type='Organization', properties={'name': 'Rbi'}), target=Node(id='Bank Of America', type='Bank', properties={'name': 'Bank Of America'}), type='imposes penalty', properties={'amount': 'Rs 10,000', 'violation': "Reserve Bank of India's instructions on reporting requirements under Liberalised Remittance Scheme 

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:21<00:07,  7.93s/it]

nodes=[Node(id='Www.Oliveboard.In', type='Website', properties={'name': 'Www.Oliveboard.In'}), Node(id='Rbi', type='Organization', properties={'name': 'Rbi'}), Node(id='Rs2000', type='Currency', properties={'name': 'Rs2000'}), Node(id='May 19, 2023', type='Date', properties={'name': 'May 19, 2023'}), Node(id='Rs3.56 Lakh Crore', type='Amount', properties={'name': 'Rs3.56 Lakh Crore'}), Node(id='November 30, 2023', type='Date', properties={'name': 'November 30, 2023'}), Node(id='Rs9,760 Crore', type='Amount', properties={'name': 'Rs9,760 Crore'}), Node(id='May 19, 2023', type='Date', properties={'name': 'May 19, 2023'}), Node(id='September 30, 2023', type='Date', properties={'name': 'September 30, 2023'}), Node(id='October 07, 2023', type='Date', properties={'name': 'October 07, 2023'}), Node(id='Tsi', type='Company', properties={'name': 'Tsi'}), Node(id='Piramal Alternatives', type='Company', properties={'name': 'Piramal Alternatives'}), Node(id='Cdpq', type='Organization', properties=

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:56<00:00, 14.05s/it]


In [80]:
# Number of nodes
print(len(graph_document.nodes))

29

In [81]:
#Number or Relationships
print(len(graph_document.relationships))

25

Reference Document: https://bratanic-tomaz.medium.com/constructing-knowledge-graphs-from-text-using-openai-functions-096a6d010c17